In [13]:
!pip install xgboost

In [14]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


In [15]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [16]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [17]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [20]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "Decision Tree classifier", 
        {"n_estimators": 19,
        "max_depth": 19,
        "max_features": "sqrt"},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Support Vector Classifier", 
        {"C": 1,
        "kernel":"rbf"},
        SVC(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBclassifier", 
        {"n_estimators":19,
        "max_depth":18,
        "learning_rate":0.7},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    )
        
]

In [21]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [ ]:
!pip install mlflow
!pip install dagshub

In [22]:

import mlflow
import mlflow.sklearn

import dagshub
dagshub.init(repo_owner='SreejithHari1881', repo_name='my-first-repo', mlflow=True)

Initialized MLflow to track repo "SreejithHari1881/my-first-repo"

Repository SreejithHari1881/my-first-repo initialized!

In [23]:
# Initialize MLflow
mlflow.set_experiment("dagshub_new")
# mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

     
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/08/22 19:23:10 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/22 19:23:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: https://dagshub.com/SreejithHari1881/my-first-repo.mlflow/#/experiments/0/runs/697ea9db37b647849cdd98d06d93fc1c.
2024/08/22 19:23:10 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/SreejithHari1881/my-first-repo.mlflow/#/experiments/0.
2024/08/22 19:23:18 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/22 19:23:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/SreejithHari1881/my-first-repo.mlflow/#/experiments/0/runs/4fba02c66497454dafddae94fe13ba76.
2024/08/22 19:23:19 INFO mlflow.tracking._tracking_service.client: 🧪 Vie